# Voting Schemes

In this project, four different voting schemes will be analysed using the TVA:

In [ ]:
import numpy as np
from abc import ABC, abstractmethod

np.random.seed(0)

Importing our happiness function and a nice printer for it.

In [ ]:
# our happiness function in python
def happiness(i: np.array, j: np.array, s: float=0.9) -> float:
    m = len(i)
    s1 = np.power([s for l in range(m)], np.power(range(m), 2))
    s2 = np.power([s for l in range(m)], np.power(m - np.array(range(m)), 2))
    d = np.abs(np.subtract(j, i))
    h_hat = np.sum(d * (s1 + s2), axis=1) / m
    return np.exp(-h_hat)

In [ ]:
def print_happiness(scheme, i, j):
    e_i = scheme._encode(i)
    e_j = scheme._encode(j)
    arr_happiness = happiness(e_i, e_j)
    mean_happiness = np.mean(arr_happiness)
    print(f"Mean happiness of our agents using this voting scheme: {np.round(mean_happiness * 100, 2)}%")
    
    for i, val in enumerate(arr_happiness):
        print(f"Agent {i+1} is {np.round(val * 100, 2)}% happy.")

Defining a parent class for any scheme we will implement later.

In [ ]:
class VotingScheme(ABC):
    def __init__(self, mapping=None):
        self._mapping = np.asarray(mapping)
        
    @abstractmethod
    def compute_res(self, preferences)->list:
        raise NotImplemented
        
    def _encode(self, ranking: np.array) -> np.array:
        preferences = ranking.copy()
        
        d = {self._mapping[n]: n for n in range(self._mapping.shape[0])}
        for mapping in self._mapping:
            preferences = np.where(preferences!=mapping, preferences, d[mapping])
        
        return preferences.astype(int)

    def _decode(self, ranking: np.array) -> np.array:
        return self._mapping[ranking]

'Test' our base class.

In [ ]:
class TestScheme(VotingScheme):
    def compute_res(self, p):
        pass

In [ ]:
preferences = np.array([
    ["Paul", "Peter", "Pia", "Patricia"],
    ["Pia", "Patricia", "Paul", "Peter"]
])

scheme = TestScheme(mapping=["Paul", "Peter", "Pia", "Patricia"])
encoded = scheme._encode(preferences)
encoded

array([[0, 1, 2, 3],
       [2, 3, 0, 1]])

In [ ]:
scheme._decode(encoded)

array([['Paul', 'Peter', 'Pia', 'Patricia'],
       ['Pia', 'Patricia', 'Paul', 'Peter']], dtype='<U8')

## 1. Voting for one

### Definition

Voters can vote for one candidate. The candidate with the most votes wins.

### Voting Vector

\{1, 0, ..., 0\}

### Implementation

In [ ]:
class VotingForOne(VotingScheme):
    def compute_res(self, preferences: np.array):
        # get the voted candidates and how often they were voted
        (unique, counts) = np.unique(preferences[:, 0], return_counts=True)
        frequencies = np.asarray((unique, counts)).T
                
        max_vote = max(counts)
        counts = [max_vote - int(l) for l in counts]
        
        frequencies = [l for l in zip(unique, counts)]
        frequencies = np.array(frequencies, dtype=np.dtype([('x', object), ('y', int)])).T
        
        # sort those candidates based on the voting count
        # if two candidates have the same counting, then sort by name
        sorted_freqs = frequencies[np.argsort(frequencies, order=('y', 'x'))]
        sorted_freqs = [l[0] for l in sorted_freqs]

        # if there are some candidates without any votes, then append them to the end of the ranking
        sorted_freqs = np.hstack((sorted_freqs, np.setxor1d(sorted_freqs, self._mapping)))
        
        return sorted_freqs
        

In [ ]:
# this voting scheme only utilizes the first column of our preferences matrix
# so basically: preferences[:, 0]
preferences = np.array([
    ["A", "B", "C", "D"],
    ["D", "A", "C", "B"],
    ["C", "A", "B", "D"],
    ["A", "C", "D", "B"]
])

In [ ]:
scheme = VotingForOne(mapping=["A", "B", "C", "D"])
outcome = scheme.compute_res(preferences)
outcome

array(['A', 'C', 'D', 'B'], dtype='<U1')

In [ ]:
print_happiness(scheme, outcome, preferences)

Mean happiness of our agents using this voting scheme: 37.73%
Agent 1 is 27.43% happy.
Agent 2 is 15.56% happy.
Agent 3 is 7.92% happy.
Agent 4 is 100.0% happy.


## 2. Voting for two

### Definition

Voters can vote for two candidates. The two candidates with the most votes win.

### Voting Vector

\{1, 1, 0, ..., 0\}

### Implementation

In [ ]:
class VotingForTwo(VotingScheme):
    def compute_res(self, preferences: np.array):
        # get the voted candidates and how often they were voted
        (unique, counts) = np.unique(preferences[:, :2], return_counts=True)
        frequencies = np.asarray((unique, counts)).T
                
        max_vote = max(counts)
        counts = [max_vote - int(l) for l in counts]
        
        frequencies = [l for l in zip(unique, counts)]
        frequencies = np.array(frequencies, dtype=np.dtype([('x', object), ('y', int)])).T
        
        # sort those candidates based on the voting count
        # if two candidates have the same counting, then sort by name
        sorted_freqs = frequencies[np.argsort(frequencies, order=('y', 'x'))]
        sorted_freqs = [l[0] for l in sorted_freqs]

        # if there are some candidates without any votes, then append them to the end of the ranking
        sorted_freqs = np.hstack((sorted_freqs, np.setxor1d(sorted_freqs, self._mapping)))
        
        return sorted_freqs
        

In [ ]:
scheme = VotingForTwo(mapping=["A", "B", "C", "D"])
outcome = scheme.compute_res(preferences)
outcome

[('A', 0) ('C', 2) ('B', 3) ('D', 3)]


array(['A', 'C', 'B', 'D'], dtype='<U1')

In [ ]:
print_happiness(scheme, outcome, preferences)

Mean happiness of our agents using this voting scheme: 29.17%
Agent 1 is 52.21% happy.
Agent 2 is 8.17% happy.
Agent 3 is 29.04% happy.
Agent 4 is 27.26% happy.


## 3. Anti-plurality voting (veto)

### Definition

In anti-plurality voting, the individual voter decides on a certain option it does not want to elect, thus votes for everyone but that one.

### Voting Vector

\{1,1,...,1,0\}

## Implementation

In [ ]:
class AntiPluralityVoting(VotingScheme):
    def compute_res(self, preferences: np.array):
        # get the voted candidates and how often they were voted
        (unique, counts) = np.unique(preferences[:, :-1], return_counts=True)
        frequencies = np.asarray((unique, counts)).T
                
        max_vote = max(counts)
        counts = [max_vote - int(l) for l in counts]
        
        frequencies = [l for l in zip(unique, counts)]
        frequencies = np.array(frequencies, dtype=np.dtype([('x', object), ('y', int)])).T
        
        # sort those candidates based on the voting count
        # if two candidates have the same counting, then sort by name
        sorted_freqs = frequencies[np.argsort(frequencies, order=('y', 'x'))]
        sorted_freqs = [l[0] for l in sorted_freqs]

        # if there are some candidates without any votes, then append them to the end of the ranking
        sorted_freqs = np.hstack((sorted_freqs, np.setxor1d(sorted_freqs, self._mapping)))
        
        return sorted_freqs

In [ ]:
scheme = VotingForTwo(mapping=["A", "B", "C", "D"])
outcome = scheme.compute_res(preferences)
outcome

[('A', 0) ('C', 2) ('B', 3) ('D', 3)]


array(['A', 'C', 'B', 'D'], dtype='<U1')

In [ ]:
print_happiness(scheme, outcome, preferences)

Mean happiness of our agents using this voting scheme: 29.17%
Agent 1 is 52.21% happy.
Agent 2 is 8.17% happy.
Agent 3 is 29.04% happy.
Agent 4 is 27.26% happy.


## 4. Borda voting

### Definition

Borda voting requires the voters to submit a full ranking of all candidates (= preference list).

### Voting Vector

\{𝑚−1,𝑚−2,...,1,0\} (where m is the number of alternatives)

### Implementation

In [ ]:
# same as in risk_function.ipynb

In [ ]:
class BordaVoting(VotingScheme):
    def compute_res(self, preferences: np.array):
        m = preferences.shape[-1]
        d = {}
        for i in range(1, m):
            # get the voted candidates and how often they were voted
            (unique, counts) = np.unique(preferences[:, i], return_counts=True)
            for j, c in enumerate(unique):
                d[c] = d.get(c, 0) + counts[j] * (m - i)
                
        max_vote = max(d.values())
        unique = list(d.keys())
        counts = [max_vote - int(l) for l in d.values()]
        
        frequencies = [l for l in zip(unique, counts)]
        frequencies = np.array(frequencies, dtype=np.dtype([('x', object), ('y', int)])).T
        
        # sort those candidates based on the voting count
        # if two candidates have the same counting, then sort by name
        sorted_freqs = frequencies[np.argsort(frequencies, order=('y', 'x'))]
        sorted_freqs = [l[0] for l in sorted_freqs]

        # if there are some candidates without any votes, then append them to the end of the ranking
        sorted_freqs = np.hstack((sorted_freqs, np.setxor1d(sorted_freqs, self._mapping)))
        
        return sorted_freqs

In [ ]:
scheme = BordaVoting(mapping=["A", "B", "C", "D"])
outcome = scheme.compute_res(preferences)
outcome

array(['B', 'C', 'A', 'D'], dtype='<U1')

In [ ]:
print_happiness(scheme, outcome, preferences)

Mean happiness of our agents using this voting scheme: 19.65%
Agent 1 is 27.96% happy.
Agent 2 is 7.92% happy.
Agent 3 is 28.14% happy.
Agent 4 is 14.6% happy.
